# Scannet vizualisation

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import panel as pn
import numpy as np
import pyvista as pv
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf

pn.extension('vtk')
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'

from torch_points3d.datasets.segmentation.scannet import ScannetDataset, Scannet
from torch_points3d.datasets.segmentation import IGNORE_LABEL

## Load Scannet dataset

In [ ]:
DIR = os.path.dirname(os.getcwd())
dataset_options = OmegaConf.load(os.path.join(DIR,'conf/data/segmentation/scannet.yaml'))

In [ ]:
dataset_options.data.dataroot = os.path.join(DIR,"data")
dataset = ScannetDataset(dataset_options.data)
print(dataset)

## Visualise the data

In [ ]:
def buil_cmap():
    mapping = np.linspace(IGNORE_LABEL, len(Scannet.VALID_CLASS_IDS)+1, 256)
    newcolors = np.zeros((256, 3))
    for i, raw_label in enumerate(Scannet.VALID_CLASS_IDS):
        newcolors[mapping >= i-0.5] = np.asarray(Scannet.SCANNET_COLOR_MAP[raw_label]) / 255.
    return ListedColormap(newcolors)
cmap = buil_cmap()

In [ ]:
def load_random_data(event):
    i = np.random.randint(0, len(dataset.train_dataset))
    sample = dataset.train_dataset[i]
    xyz = sample.pos.numpy()
    pl = pv.Plotter(notebook=True)
    point_cloud = pv.PolyData(xyz)
    point_cloud['label'] = sample.y.numpy()
    pl.add_points(point_cloud,cmap=cmap, clim=[-1, len(Scannet.VALID_CLASS_IDS)+1]) 
    
    pan.object = pl.ren_win
    return point_cloud

In [ ]:
pl = pv.Plotter(notebook=True)
pan = pn.panel(pl.ren_win, sizing_mode='scale_both', aspect_ratio=1,orientation_widget=True,)

In [ ]:
button = pn.widgets.Button(name='Load new model', button_type='primary')
button.on_click(load_random_data)

In [ ]:
dashboard = pn.Row(
    pn.Column('## Visualiser for forward runs',button),
    pan
)

In [ ]:
dashboard